In [1]:
import pandas as pd

In [2]:
import os

In [3]:
import glob

In [4]:
import re

In [5]:
pattern = re.compile(
    r'../results/GeneratedSparse/(?P<cells>\d+)_cells_(?P<features>\d+)_features/(?P<sparsity>\d+\.\d+)_(?P<method>\w+)_(?P<metric>\w+).csv'
)

In [6]:
import numpy as np

In [7]:
results = []

for filename in glob.glob('../results/GeneratedSparse/*_cells_*features/*_*_*.csv'):
    group = pattern.match(filename)
    
    cells = int(group.group('cells'))
    features = int(group.group('features'))
    method = group.group('method')
    metric = group.group('metric')
    sparsity = group.group('sparsity')
    
#     print(cells, features, method, metric)
    df = pd.read_csv(filename)
    
    elements_reinit = df.loc[:1, df.columns[0]]
    elements_sequential = df.loc[:, df.columns[0]]
    values = {
        'DataFile': filename,
        'metric': metric,
        'method': method,
        'wX': cells,
        'wY': features,
        'ID': f'{cells}_{features}',
        '|W|': cells * features,
        'logWX': np.log10(cells),
        'logWY': np.log10(features),
        'logMeanTime': np.log10(np.mean(elements_sequential)),
        'logMeanTimeReinit': np.log10(np.mean(elements_reinit)),
        'logMinTime': np.log10(np.min(elements_sequential)),
        'logMaxTime': np.log10(np.max(elements_sequential)),
        'minTime': np.min(elements_sequential),
        'maxTime': np.max(elements_sequential),
        'mean_reinit': np.mean(elements_reinit),
        'mean_sequential': np.mean(elements_sequential),
        'sd': np.std(elements_sequential),
        'Method': f'{metric}_{method}',
        'sparsity': sparsity,
    }
    
    for i in range(1, df.shape[0]):
        val = df.iloc[i][df.columns[0]]
        if val > 0:
            values[f'val_{i + 1}'] = val
        
    results.append(values)

/home/akhtyamovpavel/PythonEnvironments/BioInfoPython37/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log10


In [8]:
final_df = pd.DataFrame(results)

In [9]:
final_df.columns

Index(['DataFile', 'metric', 'method', 'wX', 'wY', 'ID', '|W|', 'logWX',
       'logWY', 'logMeanTime', 'logMeanTimeReinit', 'logMinTime', 'logMaxTime',
       'minTime', 'maxTime', 'mean_reinit', 'mean_sequential', 'sd', 'Method',
       'sparsity', 'val_2', 'val_3', 'val_4', 'val_5', 'val_6', 'val_7',
       'val_8', 'val_9', 'val_10', 'val_11', 'val_12', 'val_13', 'val_14',
       'val_15', 'val_16', 'val_17', 'val_18', 'val_19', 'val_20', 'val_21',
       'val_22', 'val_23', 'val_24', 'val_25'],
      dtype='object')

In [10]:
final_df.to_csv('../data/sparse_generated.csv', index=None)